Recurrent Neural Networks（RNNs）递归神经网络

RNN 的英文全称是 Recurrent Neural Networks ，即循环神经网络，是一种对序列型数据进行建模的深度模型。

$h_t$:记忆体内当前时刻存储的状态信息

$x_t$:当前时刻输入特征

$h_{t-1}$:记忆体上一时刻存储的状态信息

$y_t$:当前时刻循环核的输出


$y_t=softmax(h_t  w_{hy} + b_y)$

$h_t=tanh(x_t w_xh +h_{t-1} w_{hh}+b_h)$

RNNs训练和传统ANN训练异同点？

相同点：

    RNNs与传统ANN都使用BP误差反向传播算法。

不同点：

    RNNs网络参数W,U,V是共享的，而传统神经网络各层参数间没有直接联系。
    对于RNNs，在使用梯度下降算法中，每一步的输出不仅依赖当前步的网络，还依赖于之前若干步的网络状态。

为什么RNN 训练的时候Loss波动很大

​ 由于RNN特有的memory会影响后期其他的RNN的特点，梯度时大时小，learning rate没法个性化的调整，导致RNN在train的过程中，Loss会震荡起伏。为了解决RNN的这个问题，在训练的时候，可以设置临界值，当梯度大于某个临界值，直接截断，用这个临界值作为梯度的大小，防止大幅震荡。


RNN中为什么会出现梯度消失,如何解决？

RNN出现梯度消失和梯度爆炸主要体现在长句子中，因为在后向传播BP求导时，当前t时刻隐层输出的梯度包含了所有后续时刻激活函数导数的乘积，所以如果t越小、句子越长，就会出现问题。如果激活函数的导数特别小，累乘就会更小，则会出现梯度消失问题；反之，则是梯度爆炸问题。

RNN 所谓梯度消失的真正含义是，梯度被近距离梯度主导，导致模型难以学到远距离的依赖关系。

RNNs训练和传统ANN训练异同点？

相同点：都使用BP误差反向传播算法。
不同点：
RNNs网络参数W,U,V是共享的，而传统神经网络各层参数间没有直接联系。
对于RNNs，在使用梯度下降算法中，每一步的输出不仅依赖当前步的网络，还依赖于之前若干步的网络状态。

为什么RNN 训练的时候Loss波动很大?

由于RNN特有的memory会影响后期其他的RNN的特点，梯度时大时小，lr没法个性化的调整，导致RNN在train的过程中，Loss会震荡
起伏，为了解决RNN的这个问题，在训练的时候，可以设置临界值，当梯度大于某个临界值，直接截断，用这个临界值作为梯度的大小，防
止大幅震荡。

RNN中为什么会出现梯度消失？


梯度消失现象：累乘会导致激活函数导数的累乘，如果取tanh或sigmoid函数作为激活函数的话，那么必然是一堆小数在做乘法，结果就是
越乘越小。随着时间序列的不断深入，小数的累乘就会导致梯度越来越小直到接近于0，这就是“梯度消失“现象。
实际使用中，会优先选择tanh函数，原因是tanh函数相对于sigmoid函数来说梯度较大，收敛速度更快且引起梯度消失更慢。

$ \prod_{j=k-1}^3{\frac{\partial s_j}{\partial s_{j-1}}}=\prod_{j=k-1}^3{tanh' W} $

如何解决RNN中的梯度消失问题？

1.选取更好的激活函数，如Relu激活函数。ReLU函数的左侧导数为0，右侧导数恒为1，这就避免了“梯度消失“的发生。但恒为1的导数
容易导致“梯度爆炸“，但设定合适的阈值可以解决这个问题。
2.加入BN层，其优点：加速收敛.控制过拟合，可以少用或不用Dropout和正则。降低网络对初始化权重不敏感，且能允许使用较大的学习
率等。
3.改变传播结构，LSTM结构可以有效解决这个问题。





In [9]:
'''  
举例:设计一个4层的RNN,输入是一段中文，输出是一段英文，
假设每个中文字符用100维数据进行编码，每个隐含层的维度是20,有4个隐含层
input_size=100
hidden_size=20
num_layers=4
再假设模型已经训练好了，现在有个长度为10的句子做输入，那么
seq_len=10,batch_size=1
'''
import torch
import torch.nn as nn

input_size=100
hidden_size=20
num_layers=4

rnn=nn.RNN(input_size=input_size,hidden_size=hidden_size,num_layers=num_layers)#,number_layers=number_layers)
print("rnn",rnn)

seq_len=10
batch_size=1
x=torch.randn(seq_len,batch_size,input_size)
h0=torch.zeros(num_layers,batch_size,hidden_size)

out,h=rnn(x,h0)
print("out.shape",out.shape)
print("h.shape",h.shape)

rnn RNN(100, 20, num_layers=4)
out.shape torch.Size([10, 1, 20])
h.shape torch.Size([4, 1, 20])


In [10]:
'''
RNN做时序数据预测
假设现在有一系列3维飞机航迹数据，我们想预测接下来的航迹数据，那么可以考虑用RNN预测
。首先设计网络，每个航迹点都是3维的，所以input_size = 3，隐含层
hidden_size = 16，有一个隐含层，所以num_layers = 1。为了更好的利用数据，
下面代码实现的是这样的功能：输入第[1,15]个数据，输出第[6,21]个数据，
即往后平移5个单位的数据。
'''
import  torch
import datetime
import  numpy as np
import  torch.nn as nn
import  torch.optim as optim
from    matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong']
mpl.rcParams['axes.unicode_minus'] = False
###########################设置全局变量###################################

num_time_steps = 16    # 训练时时间窗的步长
input_size = 3          # 输入数据维度
hidden_size = 16        # 隐含层维度
output_size = 3         # 输出维度
num_layers = 1
lr=0.01
####################定义RNN类##############################################

class Net(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        super(Net, self).__init__()

        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        for p in self.rnn.parameters():
          nn.init.normal_(p, mean=0.0, std=0.001)

        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden_prev):

       out, hidden_prev = self.rnn(x, hidden_prev)
       # [b, seq, h]
       out = out.view(-1, hidden_size)
       out = self.linear(out)#[seq,h] => [seq,3]
       out = out.unsqueeze(dim=0)  # => [1,seq,3]
       return out, hidden_prev

####################初始化训练集#################################
def getdata():
    x1 = np.linspace(1,10,30).reshape(30,1)
    y1 = (np.zeros_like(x1)+2)+np.random.rand(30,1)*0.1
    z1 = (np.zeros_like(x1)+2).reshape(30,1)
    tr1 =  np.concatenate((x1,y1,z1),axis=1)
    # mm = MinMaxScaler()
    # data = mm.fit_transform(tr1)   #数据归一化
    return tr1

#####################开始训练模型#################################
def tarin_RNN(data):

    model = Net(input_size, hidden_size, num_layers)
    print('model:\n',model)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    #初始化h
    hidden_prev = torch.zeros(1, 1, hidden_size)
    l = []
    # 训练3000次
    for iter in range(3000):
        # loss = 0
        start = np.random.randint(10, size=1)[0]
        end = start + 15
        x = torch.tensor(data[start:end]).float().view(1, num_time_steps - 1, 3)
        # 在data里面随机选择15个点作为输入，预测第16
        y = torch.tensor(data[start + 5:end + 5]).float().view(1, num_time_steps - 1, 3)

        output, hidden_prev = model(x, hidden_prev)
        hidden_prev = hidden_prev.detach()

        loss = criterion(output, y)
        model.zero_grad()
        loss.backward()
        optimizer.step()

        if iter % 100 == 0:
            print("Iteration: {} loss {}".format(iter, loss.item()))
            l.append(loss.item())


    ##############################绘制损失函数#################################
    plt.plot(l,'r')
    plt.xlabel('训练次数')
    plt.ylabel('loss')
    plt.title('RNN损失函数下降曲线')

    return hidden_prev,model
#############################预测#########################################

def RNN_pre(model,data,hidden_prev):
    data_test = data[19:29]
    data_test = torch.tensor(np.expand_dims(data_test, axis=0),dtype=torch.float32)

    pred1,h1 = model(data_test,hidden_prev )
    print('pred1.shape:',pred1.shape)
    pred2,h2 = model(pred1,hidden_prev )
    print('pred2.shape:',pred2.shape)
    pred1 = pred1.detach().numpy().reshape(10,3)
    pred2 = pred2.detach().numpy().reshape(10,3)
    predictions = np.concatenate((pred1,pred2),axis=0)
    # predictions= mm.inverse_transform(predictions)
    print('predictions.shape:',predictions.shape)

    #############################预测可视化########################################

    fig = plt.figure(figsize=(9, 6))
    ax = Axes3D(fig)
    ax.scatter3D(data[:, 0],data[:, 1],data[:,2],c='red')
    ax.scatter3D(predictions[:,0],predictions[:,1],predictions[:,2],c='y')
    ax.set_xlabel('X')
    ax.set_xlim(0, 8.5)
    ax.set_ylabel('Y')
    ax.set_ylim(0, 10)
    ax.set_zlabel('Z')
    ax.set_zlim(0, 4)
    plt.title("RNN航迹预测")
    plt.show()

def main():
    data = getdata()
    start = datetime.datetime.now()
    hidden_pre, model = tarin_RNN(data)
    end = datetime.datetime.now()
    print('The training time: %s' % str(end - start))
    plt.show()
    RNN_pre(model, data, hidden_pre)
if __name__ == '__main__':
    main()




model:
 Net(
  (rnn): RNN(3, 16, batch_first=True)
  (linear): Linear(in_features=16, out_features=3, bias=True)
)
Iteration: 0 loss 13.739197731018066
Iteration: 100 loss 0.8503149151802063
Iteration: 200 loss 1.0509717464447021
Iteration: 300 loss 0.8992526531219482
Iteration: 400 loss 0.08515650779008865
Iteration: 500 loss 0.006117912940680981
Iteration: 600 loss 0.0017799311317503452
Iteration: 700 loss 0.002400376135483384
Iteration: 800 loss 0.0011865826090797782
Iteration: 900 loss 0.0005421483656391501
Iteration: 1000 loss 0.003756233723834157
Iteration: 1100 loss 0.0009136120206676424
Iteration: 1200 loss 0.00035660137655213475
Iteration: 1300 loss 0.00041771383257582784
Iteration: 1400 loss 0.0009116978035308421
Iteration: 1500 loss 0.000408749416237697
Iteration: 1600 loss 0.0003605236706789583
Iteration: 1700 loss 0.00035006814869120717
Iteration: 1800 loss 0.00048723191139288247
Iteration: 1900 loss 0.00035314919659867883
Iteration: 2000 loss 0.000793592946138233
Iteratio

: 

: 

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
 
# 1. 定义RNN的参数
HIDDEN_SIZE = 30                            # LSTM中隐藏节点的个数。
NUM_LAYERS = 2                              # LSTM的层数。
TIMESTEPS = 10                              # 循环神经网络的训练序列长度。
TRAINING_STEPS = 10000                      # 训练轮数。
BATCH_SIZE = 32                             # batch大小。
TRAINING_EXAMPLES = 10000                   # 训练数据个数。
TESTING_EXAMPLES = 1000                     # 测试数据个数。
SAMPLE_GAP = 0.01                           # 采样间隔。
 
 
# 2. 产生正弦数据函数
def generate_data(seq):
    X = []
    y = []
    # 序列的第i项和后面的TIMESTEPS-1项合在一起作为输入；第i + TIMESTEPS项作为输出。
    # 即用sin函数前面的TIMESTEPS个点的信息，预测第i + TIMESTEPS个点的函数值。
    for i in range(len(seq) - TIMESTEPS):
        X.append([seq[i: i + TIMESTEPS]])
        y.append([seq[i + TIMESTEPS]])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)  
 
 
# 3. 定义网络结构和优化步骤
def lstm_model(X, y, is_training):
    # 使用多层的LSTM结构。
    cell = tf.nn.rnn_cell.MultiRNNCell([
        tf.nn.rnn_cell.LSTMCell(HIDDEN_SIZE)
        for _ in range(NUM_LAYERS)]) 
 
    # 使用TensorFlow接口将多层的LSTM结构连接成RNN网络并计算其前向传播结果。
    outputs, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    # outputs是顶层LSTM在每一步的输出结果，它的维度是[batch_size, time ,
    # HIDDEN_SIZE]。在本问题中只关注最后一个时刻的输出结果。
    output = outputs[:, -1, :]
 
    # 对LSTM网络的输出再做加一层全链接层并计算损失。注意这里默认的损失为平均
    # 平方差损失函数。
    predictions = tf.contrib.layers.fully_connected(
        output, 1, activation_fn=None)
    
    # 只在训练时计算损失函数和优化步骤。测试时直接返回预测结果。
    if not is_training:
        return predictions, None, None
        
    # 计算损失函数。
    loss = tf.losses.mean_squared_error(labels=y, predictions=predictions)
 
    # 创建模型优化器并得到优化步骤。
    train_op = tf.contrib.layers.optimize_loss(
        loss, tf.train.get_global_step(),
        optimizer="Adagrad", learning_rate=0.1)
    
    return predictions, loss, train_op
 
 
# 4. 定义训练方法
def train(sess, train_X, train_Y):
    # 将训练数据以数据集的方式提供给计算图
    ds = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
    ds = ds.repeat().shuffle(1000).batch(BATCH_SIZE)   #maxi:这种该怎么理解
    X, y = ds.make_one_shot_iterator().get_next()
    
    # 定义模型，得到预测结果、损失函数，和训练操作。
    with tf.variable_scope("model"):
        _, loss, train_op = lstm_model(X, y, True)
        
    sess.run(tf.global_variables_initializer())
    for i in range(TRAINING_STEPS):
        _, l = sess.run([train_op, loss])
        if i % 1000 == 0:
            print("train step: " + str(i) + ", loss: ", str(l))
            
 
# 5. 定义测试方法
def run_eval(sess, test_X, test_y):
    # 将测试数据以数据集的方式提供给计算图。
    ds = tf.data.Dataset.from_tensor_slices((test_X, test_y))
    ds = ds.batch(1)
    X, y = ds.make_one_shot_iterator().get_next()
    
    # 调用模型得到计算结果。这里不需要输入真实的y值。
    with tf.variable_scope("model", reuse=True):
        prediction, _, _ = lstm_model(X, [0.0], False)
    
    # 将预测结果存入一个数组。
    predictions = []
    labels = []
    for i in range(TESTING_EXAMPLES):
        p, l = sess.run([prediction, y])
        predictions.append(p)
        labels.append(l)
 
    # 计算rmse作为评价指标。
    predictions = np.array(predictions).squeeze()
    labels = np.array(labels).squeeze()
    rmse = np.sqrt(((predictions - labels) ** 2).mean(axis=0))
    print("Root Mean Square Error is: %f" % rmse)
    
    # 对预测的sin函数曲线进行绘图。
    plt.figure()
    plt.plot(predictions, label='predictions')
    plt.plot(labels, label='real_sin')
    plt.legend()
    plt.show()
    
    
# 6. 生成数据并训练、验证
# 用正弦函数生成训练和测试数据集合。
# numpy.linspace函数可以创建一个等差序列的数组，它常用的参数有三个参数，
# 第一个参数表示起始值，第二个参数表示终止值，第三个参数表示数列的长度。
# 例如linespace(1, 10, 10)产生的数组是arrray([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) 
test_start = (TRAINING_EXAMPLES + TIMESTEPS) * SAMPLE_GAP
test_end = test_start + (TESTING_EXAMPLES + TIMESTEPS) * SAMPLE_GAP
train_X, train_y = generate_data(np.sin(np.linspace(
    0, test_start, TRAINING_EXAMPLES + TIMESTEPS, dtype=np.float32)))
test_X, test_y = generate_data(np.sin(np.linspace(
    test_start, test_end, TESTING_EXAMPLES + TIMESTEPS, dtype=np.float32)))
 
#maxi:可以看到最后的步骤就是先训练模型，然后把训练出的模型拿去预测
with tf.Session() as sess:
    train(sess, train_X, train_y)
    run_eval(sess, test_X, test_y)

AttributeError: module 'tensorflow' has no attribute 'Session'

In [5]:
#https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial

from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

#print(findFiles('data/names/*.txt'))
print(findFiles('E:\坚果云\精益求精\warmup\RNNLSTMGRU\data\datanames\*.txt'))
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)


[]
Slusarski


In [4]:
print(category_lines['Italian'][:5])

KeyError: 'Italian'